<h3> Segmenting and Clustering Neighborhoods in Toronto</h3>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import io

WEB SCRAPING

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
web_texto= requests.get(wikipedia_link).text


Canada_dado = BeautifulSoup(web_texto,'xml')

EXTRAINDO DADOS DA WEBPAGE

In [3]:
table = Canada_dado.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []


for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)




INTEGRANDO AS VIZINHANÇAS 

In [4]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


CRIANDO O DATAFRAME

In [5]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df = pd.DataFrame.from_dict(toronto_dict)

df.head(14)

,Postcode,Borough,Neighbourhood
0,M4H,East York,Thorncliffe Park
1,M5E,Downtown Toronto,Berczy Park
2,M5M,North York,Bedford Park
3,M1S,Scarborough,Agincourt
4,M6K,West Toronto,"Exhibition Place, Parkdale Village"
5,M8Y,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que..."
6,M9M,North York,"Emery, Humberlea"
7,M3J,North York,"Northwood Park, York University"
8,M6S,West Toronto,"Runnymede, Swansea"
9,M7A,Downtown Toronto,Queen's Park


LATITUDE E LONGITUDE

In [6]:
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [7]:
c.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
c.columns = ['Postcode', 'Latitude', 'Longitude']

In [9]:
df_toronto = pd.merge(c, df, on='Postcode')

In [10]:
df_toronto = df_toronto[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]

In [29]:
df_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
6,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
7,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476
8,M1N,Scarborough,Birch Cliff,43.692657,-79.264848
9,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
